In [1]:
#| default_exp routes.slack

# Slack Routes
> route functions for interacting with Slack via slackbolt

## To create a slackbot
1. Go to your Slack App settings (api.slack.com/apps)
2. Under "Event Subscriptions", enable events
3. Subscribe to these bot events:
   - message.channels
   - app_mentions

4. Also verify that your bot has been:
   - Added to the channel you're testing in
   - Given the necessary OAuth scopes:
      - chat:write
      - app_mentions:read
      - channels:history

5. Deploy bot to a channel and 
6. update .env with requisite SLACK_BOT_TOKEN, SLACK_SIGNING_SECRET, SLACK_APP_TOKEN

In [2]:
#| export
from slack_bolt.async_app import AsyncApp as AsyncSlackApp
from slack_sdk.errors import SlackApiError

from agent_mafia.client.ResponseGetData import ResponseGetDataSlack

In [3]:
#| exporti
import datetime as dt
import os
from nbdev.showdoc import patch_to

from agent_mafia.client.MafiaError import MafiaError 

In [4]:
#| hide 
import nbdev

In [5]:
default_slack_app = AsyncSlackApp(
    token=os.environ["SLACK_BOT_TOKEN"],
    signing_secret=os.environ["SLACK_SIGNING_SECRET"],
)

channel_id = "C08JLCQQYAU"

In [6]:
#| exporti

@patch_to(AsyncSlackApp)
async def who_am_i(self):
    res = await self.client.auth_test()

    return ResponseGetDataSlack.from_res(res = res, async_app = self)


In [7]:
await default_slack_app.who_am_i()

SlackApiError: The request to the Slack API failed. (url: https://slack.com/api/auth.test, status: 200)
The server responded with: {'ok': False, 'error': 'invalid_auth'}

In [ ]:
#| export

async def get_channel_history(async_app: AsyncSlackApp, channel_id: str, return_raw: bool = False):
    
    try:
        # Call the conversations.history method using the WebClient
        # conversations.history returns the first 100 messages by default
        # These results are paginated, see: https://api.slack.com/methods/conversations.history$pagination
        res = await async_app.client.conversations_history(channel=channel_id)

        rgd = ResponseGetDataSlack.from_res(res = res, async_app = async_app)

        if return_raw:
            return rgd
        
        rgd.response = res.data['messages']
        return rgd

    except SlackApiError as e:
        raise MafiaError(exception = e, message = "Error fetching conversation history")


In [ ]:
(await get_channel_history(default_slack_app, channel_id=channel_id)).response

[{'user': 'U08J0GT9QAX',
  'type': 'message',
  'ts': '1742237263.728579',
  'edited': {'user': 'B08J0GT9G8K', 'ts': '1742237263.000000'},
  'bot_id': 'B08J0GT9G8K',
  'app_id': 'A08J92PTA7K',
  'text': 'Hello, World! updated 2025-03-17 18:47:43.824275',
  'team': 'T012H3JSC83',
  'bot_profile': {'id': 'B08J0GT9G8K',
   'deleted': False,
   'name': 'Emma_v2',
   'updated': 1742171723,
   'app_id': 'A08J92PTA7K',
   'icons': {'image_36': 'https://avatars.slack-edge.com/2025-03-16/8612624392195_32caa386e2553c6b47d2_36.webp',
    'image_48': 'https://avatars.slack-edge.com/2025-03-16/8612624392195_32caa386e2553c6b47d2_48.webp',
    'image_72': 'https://avatars.slack-edge.com/2025-03-16/8612624392195_32caa386e2553c6b47d2_72.webp'},
   'team_id': 'T012H3JSC83'},
  'blocks': [{'type': 'rich_text',
    'block_id': '36s',
    'elements': [{'type': 'rich_text_section',
      'elements': [{'type': 'text',
        'text': 'Hello, World! updated 2025-03-17 18:47:43.824275'}]}]}]},
 {'user': 'U08J0

In [ ]:
# | export
async def post_message(
    async_app: AsyncSlackApp,
    channel_id: str,
    new_text: str,
    return_raw: bool = False
):
    try:
        res = await async_app.client.chat_postMessage(channel=channel_id, text=new_text)

        rgd = ResponseGetDataSlack.from_res(res=res, async_app=async_app, message_id = res.data['ts'])

        if return_raw:
            return rgd
    
        rgd.response = rgd.response['message']
        return rgd

    except SlackApiError as e:
        raise MafiaError(message="Error posting message", exception=e) from e

In [ ]:
res = await post_message(async_app=default_slack_app, channel_id=channel_id, new_text="Hello, world!")
res.response

{'user': 'U08J0GT9QAX',
 'type': 'message',
 'ts': '1742238810.436289',
 'bot_id': 'B08J0GT9G8K',
 'app_id': 'A08J92PTA7K',
 'text': 'Hello, world!',
 'team': 'T012H3JSC83',
 'bot_profile': {'id': 'B08J0GT9G8K',
  'app_id': 'A08J92PTA7K',
  'name': 'Emma_v2',
  'icons': {'image_36': 'https://avatars.slack-edge.com/2025-03-16/8612624392195_32caa386e2553c6b47d2_36.webp',
   'image_48': 'https://avatars.slack-edge.com/2025-03-16/8612624392195_32caa386e2553c6b47d2_48.webp',
   'image_72': 'https://avatars.slack-edge.com/2025-03-16/8612624392195_32caa386e2553c6b47d2_72.webp'},
  'deleted': False,
  'updated': 1742171723,
  'team_id': 'T012H3JSC83'},
 'blocks': [{'type': 'rich_text',
   'block_id': 'acN',
   'elements': [{'type': 'rich_text_section',
     'elements': [{'type': 'text', 'text': 'Hello, world!'}]}]}]}

In [ ]:
# | export


async def update_slack_message(
    async_app: AsyncSlackApp,
    channel_id: str,  # The ID of the channel where the message is located.
    message_id: str,  # The timestamp of the message to update.
    new_text,  # str The new text content for the message.
    return_raw: bool = False,
):
    """
    Updates an existing Slack message.
    """
    try:
        res = await async_app.client.chat_update(
            channel=channel_id, ts=message_id, text=new_text
        )

        rgd = ResponseGetDataSlack.from_res(
            res=res,
            async_app=async_app,
            message_id=res.data["ts"],
            channel_id=res.data["channel"],
        )

        if return_raw:
            return rgd

        rgd.response = rgd.response["message"]

        return rgd

    except SlackApiError as e:
        raise MafiaError(exception=e) from e

In [ ]:
await update_slack_message(
    default_slack_app,
    channel_id=channel_id,
    message_id=res.message_id,
    new_text=f"Hello, World! updated {dt.datetime.now()}",
)

ResponseGetDataSlack(is_success=True, status=200, response={'user': 'U08J0GT9QAX', 'type': 'message', 'edited': {'user': 'B08J0GT9G8K', 'ts': '1742238810.000000'}, 'bot_id': 'B08J0GT9G8K', 'app_id': 'A08J92PTA7K', 'text': 'Hello, World! updated 2025-03-17 19:13:30.493061', 'team': 'T012H3JSC83', 'bot_profile': {'id': 'B08J0GT9G8K', 'app_id': 'A08J92PTA7K', 'name': 'Emma_v2', 'icons': {'image_36': 'https://avatars.slack-edge.com/2025-03-16/8612624392195_32caa386e2553c6b47d2_36.webp', 'image_48': 'https://avatars.slack-edge.com/2025-03-16/8612624392195_32caa386e2553c6b47d2_48.webp', 'image_72': 'https://avatars.slack-edge.com/2025-03-16/8612624392195_32caa386e2553c6b47d2_72.webp'}, 'deleted': False, 'updated': 1742171723, 'team_id': 'T012H3JSC83'}, 'blocks': [{'type': 'rich_text', 'block_id': 'Inb', 'elements': [{'type': 'rich_text_section', 'elements': [{'type': 'text', 'text': 'Hello, World! updated 2025-03-17 19:13:30.493061'}]}]}]}, channel_id='C08JLCQQYAU', message_id='1742238810.43

In [ ]:
#| export
async def delete_slack_message(
    async_app: AsyncSlackApp,
    channel_id: str, # The ID of the channel where the message is located.
    message_id: str,  # The timestamp of the message to update.
    return_raw : bool = False
):
    """
    Delete an existing Slack message.
    """
    try:
        res = await async_app.client.chat_delete(channel=channel_id, ts=message_id)

        rgd = ResponseGetDataSlack.from_res(res=res, async_app=async_app, message_id = res.data['ts'],
                                            channel_id = res.data['channel'])

        if return_raw:
            return rgd

        rgd.response = res['ok']

                                            
        return rgd
        
    except SlackApiError as e:
        raise MafiaError(exception = e) from e

In [ ]:
await delete_slack_message(
    default_slack_app,
    channel_id=channel_id,
    message_id=res.message_id,
)

ResponseGetDataSlack(is_success=True, status=200, response=True, channel_id='C08JLCQQYAU', message_id='1742238810.436289')

In [ ]:
#| hide
nbdev.nbdev_export('./slack.ipynb')